# Технезис, Junior аналитик данных, Тестовое задание

Подготовил: Фролов Константин

Дата последнего обновления: 26.10.2023

## ТЗ

**Задание**

Необходимо обработать сырые данные поступившие от парсера.

**Описание данных**

В файле hotels_parsing_result.xlsx содержатся данные о гостиницах от двух разных платформ в одном регионе (вкладки platform1, platform2).

| Поле | Описание |
| --- | --- |
| id                                          | Идентификатор в БД                                        |
| create_time                                 | Дата создания                                            |
| title                                       | Название                                                 |
| hotel_type_original                         | Тип гостиницы                                            |
| city                                       | Город                                                    |
| address                                     | Адрес                                                    |
| rating                                     | Рейтинг                                                  |
| rating_5                                   | Рейтинг по 5-ти бальной шкале                            |
| review_count                               | Количество отзывов                                       |
| star_rating                                | Звездность                                               |
| rooms_count                                | Количество номеров                                       |
| contact_social                             | Контакты соц. сетей                                      |
| description                                | Описание                                                 |
| email                                      | email строкой, несколько значений через запятую          |
| phone                                      | телефон строкой, несколько значений через запятую       |
| website                                    | сайты строкой, несколько значений через запятую        |
| uid                                        | Уникальный идентификатор гостиницы на платформе, не может быть разным у одной гостиницы, и не может повторятся у разных гостиниц, но в рамках одной платформы |
| parsing_time                               | Время сбора                                             |
| lat                                        | Широта                                                  |
| lon                                        | Долгота                                                 |

**Состав данных**

- Записи могут повторятся, и самые актуальные данные идут в конце;
- Если какое-то поле по конкретной гостинице пустое в последней записи, но встречалось ранее, то необходимо его взять из более ранних записей;
- Поля телефона, email, сайтов могу содержать лишние символы, несколько записей и прочее;
- Формат записи названия, адреса, типа гостиницы отличается в разных платформах.


**Задача**

1. Собрать от каждой платформы финальный список гостиниц, в котором по каждой гостинице внутри платформы будет только одна запись с самыми актуальными и полными данными;
2. Почистить данные
3. Поля телефона, email, сайтов распарсить и сохранить как списки в одинаковом формате (address@domen.org, 79234553322, domen.ru)
4. Вывести топ 10 по каждой платформе, по параметрам:
  1. больше всего телефонов;
  2. больше всего отзывов.
5. Вывести квадрат координат размером 1км на 1км, где больше всего гостиниц
6. Задача со *, объединить данные от двух платформ, по критерию который вы придумаете:
  1. Вывести все гостиницы, которые есть в платформе 1 и нет в платформе 2;
  2. Вывести топ 10 гостиниц которые есть в обоих платформах, по суммарному количеству отзывов.


## Получение данных

Файл requirments.txt доступен по [ссылке](https://drive.usercontent.google.com/download?id=1KoxJTW4nS2sg0CZYkRX2vA-aA9jAmKFL&export=download&authuser=0&confirm=t&uuid=87a4e8b9-46cf-46bd-aa3a-f062186e3af5&at=APZUnTUDyVNAoonNOh0eoo1xt-kI:1698308626458).

In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.graph_objects as go

In [ ]:
url = 'https://drive.usercontent.google.com/download?id=1SKiomla8fy5tPs56i123eYcXig0Npn_3&export=download&authuser=0&confirm=t&uuid=0704ea4c-510d-4bb4-98b0-768fd4aed0a1&at=APZUnTXla0qMtmvNHBEPxmFcWAxq:1698246899680'

xls = pd.ExcelFile(url)
df1 = pd.read_excel(xls, 'platform1')
df2 = pd.read_excel(xls, 'platfrom2') # опечатка в названии вкладки

1. Посмотрим на данные из первого источника.

In [ ]:
df1.info()

In [ ]:
df1.head()

2. Посмотрим на данные из второго источника.

In [ ]:
df2.info()

In [ ]:
df2.head()

3. Посмотрим, о каком количестве гостиниц имеется информация в каждом из случаев.

In [ ]:
df1['uid'].nunique()

In [ ]:
df2['uid'].nunique()

4. Первые выводы:
- Нужно привести типы данных к одному виду в обеих таблицах (касается столбцов 8, 12-16);
- Нужно преобразовать столбцы со временем для корректной работы (столбцы 1, 18).

## Предобработка данных

### Типы данных

In [ ]:
# Преобразование к общим типам
df1['review_count'] = df1['review_count'].dropna().astype('float')
df1[
    ['contact_social',
     'description',
     'email',
     'phone',
     'website']] = df1[
        ['contact_social',
         'description',
         'email',
         'phone',
         'website']
        ].astype('str')

# Преобразование столбцов со временем
for i in ['create_time', 'parsing_time']:
    df1[i] = pd.to_datetime(
        df1[i], format='%Y-%m-%d %H:%M:%S.%f' #2023-09-14 15:46:42.672665
        )
    df2[i] = pd.to_datetime(
        df1[i], format='%Y-%m-%d %H:%M:%S.%f'
        )

### Пропуски и дубликаты

In [ ]:
# Замена текстовых nan на фактические Nan значения в df1
cols = ['contact_social', 'description', 'email', 'phone', 'website']
df1[cols] = df1[cols].replace('nan', np.NaN)

In [ ]:
df1.isna().sum()

In [ ]:
df2.isna().sum()

Обе таблицы содержат пропуски значений.

Посмотрим, есть ли дубликаты значений в столбце `uid`, который содержит уникальный индентификатор гостиницы.

In [ ]:
df1['uid'].duplicated().sum()

In [ ]:
df2['uid'].duplicated().sum()

Во второй таблице много повторяющихся строк.

### Очистка данных

#### 1. Столбец hotel_type_original


In [ ]:
df1['hotel_type_original'].value_counts()

In [ ]:
df2['hotel_type_original'].value_counts()

В двух системах по-разному кодируются типы отелей.

#### 2. Столбец city

In [ ]:
df1['city'].unique()

In [ ]:
df2['city'].unique()

В данном случае важно перевести названия на русский язык.

In [ ]:
city_dict = {
    'ufa': 'Уфа',
    'magnitogorsk': 'Магнитогорск',
    'sterlitamak': 'Стерлитамак',
    'salavat': 'Салават',
    'neftekamsk': 'Нефтекамск',
    'oktyabrskiy': 'Октябрьский',
    'beloretsk': 'Белорецк',
    'izhevsk': 'Ижевск',
    'kumertau': 'Кумертау'
}
df2['city'] = df2['city'].map(city_dict)

#### 3. Столбец address

In [ ]:
df1['address'].sample(10).tolist()

In [ ]:
df2['address'].sample(10).tolist()

Замечены следующие особенности:

1. В первом датафрейме часть информации написана на английском языке, город указан последним и всегда на русском, имеются технические символы "\xa0";
2. Во втором датафрейме также имеются технические символы "\xa0".

Удалим страну, регион, городской округ и город из df2, и значение города из df1 (на последнем месте).

In [ ]:
df1['address'] = df1['address'].str.rsplit(',', n=1).str[0]
df2['address'] = df2['address'].apply(lambda x: ', '.join(x.split(', ')[4:]) if isinstance(x, str) else x[4:])

# Удаление лишних пробелов в начале и в конце строки
df2['address'] = df2['address'].str.strip()

In [ ]:
df1['address'].sample(10).tolist()

In [ ]:
df2['address'].sample(10).tolist()

Удалим технические символы, заменив их на пробелы.

In [ ]:
df1['address'] = df1['address'].str.replace('\xa0', ' ')
df2['address'] = df2['address'].str.replace('\xa0', ' ')

Для второго датафрейма можно восстановить пропущенные значения города через адрес.

In [ ]:
# Создание столбца с населенным пунктом, который получен из адреса
df2['city_gen'] = df2['address'].str.split(',').str[3].replace('', np.NaN)

In [ ]:
# Заполнение пропусков в столбце город
df2.loc[df2['city'].isna(), 'city'] = df2.loc[df2['city'].isna(), 'city_gen']

In [ ]:
# Удаление лишних пробелов в начале и в конце строки
df2['city'] = df2['city'].str.strip()
df2['city_gen'] = df2['city_gen'].str.strip()

Посмотрим, сколько городов удалось таким образом восстановить.

In [ ]:
print("Количество оставшихся пропусков в столбце city:",
      df2['city'].isna().sum())

Из 1145 осталось 150 пропусков.

In [ ]:
# Удаление технического столбца с городом
df2 = df2.drop('city_gen', axis=1)

#### 4. Столбцы rating, rating_5

In [ ]:
df1['rating'].describe()

In [ ]:
df2['rating'].describe()

In [ ]:
df1['rating_5'].describe()

In [ ]:
df2['rating_5'].describe()

Очевидно, что между датафреймами он отличается по категориям (5 или 10 баллов максимум). Для решения уже есть столбец rating_5.

#### 5. Столбец review_count

In [ ]:
df1['review_count'].describe()

In [ ]:
df2['review_count'].describe()

#### Столбец region

In [ ]:
df1['region'].value_counts()

In [ ]:
df2['region'].value_counts()

Все данные по Республике Башкортостан. Однако, выше в списке городов первого датафрейма были "Зеленоградск", "Калининград", "Светлогорск", "Балтийск", "Янтарный". Очевидно, что это ошибка. Определим такие отели и заменим в них номер региона.

In [ ]:
# Координаты Москвы
moscow_longitude = 37.6176

# Если западнее, то сменить регион на 39
df1['region'] = np.where(df1['lon'] < moscow_longitude, 39, df1['region'])

In [ ]:
df1['region'].value_counts()

#### 6. Столбец star_rating

In [ ]:
df1['star_rating'].describe()

In [ ]:
df2['star_rating'].describe()

В случае со вторым датафреймом количество звезд не превышает 4.

#### 7. Столбец rooms_count

In [ ]:
df1['rooms_count'].describe()

In [ ]:
df2['rooms_count'].describe()

#### Столбец contact_social

В первом датафрейме данных по столбцу нет, поэтому распарсим второй датафрейм.

In [ ]:
def extract_https(s):
    try:
        contacts = json.loads(s)
        return ', '.join([contact['contact'] for contact in contacts])
    except:
        return np.NaN

In [ ]:
df2['contact_social'] = df2['contact_social'].apply(extract_https)

### Заполнение значений

По заданию необходимо максимально заполнить данные на основании имеющихся. Воспользуемся методом `ffill` из функции `fillna()`, а затем отбросим все строки, кроме последних.

In [ ]:
# Сортировка по дате
df1 = df1.sort_values('parsing_time')
df2 = df2.sort_values('parsing_time')

# Заполнение пропущенных значений последним известным значением в каждой группе
df1 = (
    df1
    .groupby('uid', group_keys=True)
    .apply(lambda group: group.fillna(method='ffill'))
    .reset_index(drop=True)
)
df2 = (
    df2
    .groupby('uid', group_keys=True)
    .apply(lambda group: group.fillna(method='ffill'))
    .reset_index(drop=True)
)

# Сохранение только последнего значения из группы
df1 = df1.drop_duplicates(subset='uid', keep='last')
df2 = df2.drop_duplicates(subset='uid', keep='last')

In [ ]:
df1['uid'].nunique()

In [ ]:
df2['uid'].nunique()

### Столбцы email, phone, website

По заданию поля телефона, email, сайтов необходимо распарсить и сохранить как списки в одинаковом формате. Опять же обратимся только ко второму датафрейму.

In [ ]:
df2[~df2['email'].isna()]['email'].sample(15)

In [ ]:
df2['phone'].sample(15)

In [ ]:
df2['website'].sample(15)

Email уже имеют единообразный вид, в отличие от двух других столбцов.

Начнем с номера телефона.

In [ ]:
def correct_phone_num(s):

    if pd.notna(s):
        # Удаляет '+'
        s = s.replace('+', '')
        # Разделяет номера, если их больше двух в ряд
        numbers = s.split(',')
        modified_numbers = []
        for number in numbers:
            # Удаляет пробелы
            number = number.strip()
            # Проверяет размер номера и первую цифру
            if len(number) == 11 and number.startswith('8'):
                number = '7' + number[1:]
            modified_numbers.append(number)
        return ', '.join(modified_numbers)
    else:
      return s

df2['phone'] = df2['phone'].apply(correct_phone_num)


In [ ]:
df2['phone']

Теперь обработаем сайты.

In [ ]:
def extract_domain(domain):
    try:
        parts = domain.split('.')
        domains = parts[-1]
        return domains
    except:
        return ''

df2['domain'] = df2['website'].apply(extract_domain)

In [ ]:
df2['domain'].value_counts()

In [ ]:
df2[df2['domain'] == 'link'][['contact_social', 'website', 'domain']].head()

По списку доменов видно, что есть просто текст - его будем удалять, а есть подразделы сайта. Также ссылки вроде vk.link стоит удалить из столбца сайтов.

In [ ]:
df2.loc[df2['website'] == 'vk.link', 'website'] = np.NaN

In [ ]:
allowed_domains = ['.ru', '.рф', '.com', '.biz', '.site', '.su', '.camp', '.cc', '.ws', '.club', '.net', '.camp']

def extract_domain(url):
    try:
        url = url.replace('http://', '').replace('https://', '')
        # Делит домен на части
        domain, _, _ = url.partition('/')
        # Удаляет 'www.'
        if domain.startswith('www.'):
            domain = domain[4:]
        # Проверяет, находится ли домен верхнего уровня среди разрешенных
        top_level = '.' + domain.split('.')[-1]
        if top_level not in allowed_domains:
            return np.NaN
        return domain
    except:
        return np.NaN

# Apply the function to your column
df2['website'] = df2['website'].apply(extract_domain)


In [ ]:
df2['website'].value_counts().head(50)

Теперь можно создать список с email, номером телефона и адресом сайта. Создадим два списка: где есть хотя бы один элемент, и где есть сразу все три.

In [ ]:
contacts = df2[['email', 'phone', 'website']]
contacts = contacts.dropna(how='all')
contacts_list = contacts.values


In [ ]:
contacts = df2[['email', 'phone', 'website']]
contacts = contacts.dropna(how='any')
contacts_list_full = contacts.values

### Топ-10 по каждой платформе

Выведем топ 10 по каждой платформе, по параметрам:

  - больше всего телефонов;
  - больше всего отзывов.

1. В случае с первой платформой телефонов, как и других данных о контактах, не оказалось. Далее будут отображены топ-10 для второй платформы.

In [ ]:
# Подсчет количества телефонов в строке
df2['phone_count'] = df2[df2['phone'].notna()]['phone'].str.split(',').apply(len)

df2_sorted = df2.sort_values('phone_count', ascending=False)
df2_sorted.head(10)


В топ-10 попали отели с 3-6 номерами.

2. Топ-10 по количеству отзывов для двух платформ.

In [ ]:
df1_sorted = df1.sort_values('review_count', ascending=False)
df2_sorted = df2.sort_values('review_count', ascending=False)

In [ ]:
df1_sorted.head(10)

In [ ]:
df2_sorted.head(10)

In [ ]:
platform1 = df1_sorted['review_count'].head(10).tolist()
platform2 = df2_sorted['review_count'].head(10).tolist()
index = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
df_top = pd.DataFrame({'Платформа 1': platform1,
                       'Платформа 2': platform2}, index=index)
ax = df_top.plot.bar(rot=0)

На графике видно, что в топ-10 первое место досталось первой платформе.

### Квадрат с наибольшим количеством гостиниц

Посмотрим на карту гостиниц.

In [ ]:
fig = go.Figure(go.Densitymapbox(lat=df1['lat'], lon=df1['lon'],
                                 radius=10))
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = go.Figure(go.Densitymapbox(lat=df2['lat'], lon=df2['lon'],
                                 radius=10))
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Очевидно, что на первой платформе есть гостиницы, которые находятся за пределами Республики Башкортостан, в отличие от второй платформы. В целом, Уфа и Калининград соперничают за первое место в категории плотности отелей на квадратный километр.

### Объединение таблиц

Объединение таблиц без id - непростая задача. Попробуем использовать геолокацию как точку отсчета.

1. Выведем все гостиницы, которые есть в платформе 1 и нет в платформе 2.

In [ ]:
df3 = df1.merge(df2, on=['lat', 'lon'], how='left')

In [ ]:
df3[['lat', 'lon']].duplicated().sum()

In [ ]:
df3 = df3.drop_duplicates(subset=['lat', 'lon'])

In [ ]:
df3

2. Выдем топ 10 гостиниц которые есть в обоих платформах, по суммарному количеству отзывов.

In [ ]:
df4 = df1.merge(df2, on=['lat', 'lon'], how='inner')
df4

По геолокации общие отели найти не удалось.

Объединение таблиц по названию, к сожалению, также дает неполную картину.